In [1]:
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py

--2024-06-29 17:01:23--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py
Connecting to 127.0.0.1:3128... connected.
Proxy request sent, awaiting response... 200 OK
Length: 3832 (3.7K) [text/plain]
Saving to: ‘minsearch.py.2’

minsearch.py.2      100%[===================>]   3.74K  --.-KB/s    in 0.04s   

2024-06-29 17:01:24 (84.3 KB/s) - ‘minsearch.py.2’ saved [3832/3832]



In [2]:
!wget https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/01-intro/documents.json

--2024-06-29 17:01:24--  https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/01-intro/documents.json
Connecting to 127.0.0.1:3128... connected.
Proxy request sent, awaiting response... 200 OK
Length: 658332 (643K) [text/plain]
Saving to: ‘documents.json.2’

documents.json.2    100%[===================>] 642.90K  2.00MB/s    in 0.3s    

2024-06-29 17:01:25 (2.00 MB/s) - ‘documents.json.2’ saved [658332/658332]



In [3]:
import minsearch

In [4]:
import json

In [5]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [6]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [7]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [8]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

SELECT * WHERE course = 'data-engineering-zoomcamp';

In [9]:
q = 'the course has already started, can I still enroll?'

In [10]:
index.fit(documents)

In [11]:
# from openai import OpenAI
from groq import Groq

from dotenv import load_dotenv

load_dotenv()

True

In [12]:
# client = OpenAI()
client = Groq()

In [13]:
response = client.chat.completions.create(
    # model='gpt-4o',
    model="llama3-8b-8192",
    messages=[{"role": "user", "content": q}]
)

# print(response.choices[0].message.content)

In [14]:
print(response.choices[0].message.content)

I'm happy to help you with your question!

The answer to this depends on the specific course and institution you're interested in enrolling in. Here are a few possibilities:

1. **Check with the course administrator**: Reach out to the course administrator or instructor to ask about the possibility of still enrolling. They can let you know if there's still space available and what the process is for late enrollment.
2. **Check the course website or syllabus**: Review the course website or syllabus to see if there's a deadline for enrollment. If the deadline has already passed, you might not be able to enroll.
3. **Check with the institution's student services**: Contact the institution's student services or registrar's office to ask about their policies for late enrollment. They can let you know if there are any options available.
4. **Consider alternative courses or programs**: If you're unable to enroll in the specific course you're interested in, you might want to look into alternat

In [15]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results

In [16]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [17]:
def llm(prompt):
    response = client.chat.completions.create(
        # model='gpt-4o',
        model="llama3-8b-8192",
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [18]:
query = 'how do I run kafka?'

def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [19]:
rag(query)

'Based on the context, I will answer your question.\n\nThe context is related to running Kafka in a terminal. The questions and answers provide instructions on how to run specific Kafka-related commands or scripts.\n\nTo run Kafka, you can follow the instructions in the context. Here are a few options:\n\n* If you want to run a Java Kafka producer, consumer, or KStreams, you need to run the following command in the project directory:\n`java -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java`\n\n* If you\'re trying to run a Python file related to Kafka and getting a "Module \'kafka\' not found" error, you need to create a virtual environment and install the required packages. You can do this by running the following commands:\n```\npython -m venv env\nsource env/bin/activate\npip install -r ../requirements.txt\n```\n* If you\'re getting a permission denied error while running a build script, you can fix it by running the following command:\n`chmod

In [20]:
print(rag('the course has already started, can I still enroll?'))

Based on the provided context, the QUESTION is: the course has already started, can I still enroll?

According to the CONTEXT, it is possible to join the course after the start date. The answer to the question "Can I still join the course after the start date?" is not explicitly mentioned, but we can infer that the answer is yes based on the fact that even if someone doesn't register, they are still eligible to submit the homeworks.


In [21]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [22]:
from elasticsearch import Elasticsearch

In [23]:
es_client = Elasticsearch('http://localhost:9200') 

In [24]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

BadRequestError: BadRequestError(400, 'resource_already_exists_exception', 'index [course-questions/kzOvq0oRQeOSpvZUGfuPUg] already exists')

In [ ]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [ ]:
from tqdm.auto import tqdm

/home/erstome/LLM-zoomcamp-2024/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|██████████| 948/948 [00:06<00:00, 143.41it/s]


In [ ]:
query = 'I just disovered the course. Can I still join it?'

In [ ]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

In [ ]:
def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [ ]:
print(rag(query))

Thank you for your interest in the course! According to our FAQ, yes, you can still join the course even after the start date. You're eligible to submit homeworks, and while there will be deadlines for final projects, you don't need to rush.
